In [1]:
import re
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer

In [14]:
DATA_IN_PATH = './data_in/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA, header = 0 , quoting = 3)


train_data

,review,sentiment
0,stuff going moment mj started listening music ...,1
1,classic war worlds timothy hines entertaining ...,1
2,film starts manager nicholas bell giving welco...,0
3,must assumed praised film greatest filmed oper...,0
4,superbly trashy wondrously unpretentious explo...,1
5,dont know people think bad movie got pretty go...,1
6,movie could good comes way short cheesy specia...,0
7,watched video friend house glad waste money bu...,0
8,friend mine bought film even grossly overprice...,0
9,movie full references like mad max ii wild one...,1


In [15]:
reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

In [16]:
# TF-IDF 벡터화
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df = 0.0 , analyzer = 'char', sublinear_tf = True ,
                             ngram_range=(1,3), max_features = 5000)

X = vectorizer.fit_transform(reviews)

"""
min_df : 설정한 값보다 특정 토큰의 df 값이 더 적게 나오면 벡터화 과정에서 제거한다는 의미
analyzer : 분석하기 위한 기준 단위 ('word' 와 'char'로 2가지 옵션을 제공
                                     word : 단어 하나를 단위로 하는 것
                                     char : 문자 하나를 단위로 하는 것)
sublinear_tf : 문서의 단어 빈도 수에 대한 스무딩 여부를 설정하는 값
max_features : 각 벡터의 최대 길이, 특징의 길이를 설정하는 것
"""

In [17]:
# 학습과 검증 데이터셋 분리

from sklearn.model_selection import train_test_split
import numpy as np

RANDOM_SEED = 42
TEST_SPLIT = 0.2

y = np.array(sentiments)

X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size = TEST_SPLIT, random_state = RANDOM_SEED)

In [18]:
# 모델 선언 및 학습

# 선형 회귀 모델을 만들기 위해 사이킷런 라이브러리에서 지원하는 LogisticRegression 클래스의 객체를 생성
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(class_weight = 'balanced')
lgs.fit(X_train, y_train)

# class_weight을 'balanced'로 설정해서 각 라벨에 대해 균형 있게 학습할 수 있게 한 것

C:\Users\user\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [27]:
# 검증 데이터로 성능 평가
print("Accuracy:", lgs.score(X_eval, y_eval)) # 검증 데이터로 성능 측정

"""
성능 평가 방법에는 다양한 방법이 있다.
정확도(Accuracy), 정밀도(Precision), 재현율(recall), f1-score, auc 등
"""

Accuracy: 0.8596


In [32]:
# 데이터 제출

TEST_CLEAN_DATA = 'test_clean.csv'

test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA, header = 0, quoting = 3)

In [33]:
testDataVecs = vectorizer.transform(test_data['review'])

In [34]:
test_predicted = lgs.predict(testDataVecs)
print(test_predicted)

[1 0 1 ... 0 1 0]


In [36]:
import os

DATA_OUT_PATH = './data_out/'

if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)
    
ids = list(test_data['id'])
answer_dataset = pd.DataFrame({'id':ids, 'sentiment':test_predicted})
answer_dataset.to_csv(DATA_OUT_PATH + 'lgs_tfidf_answer.csv')